# **Гибридная нейронная сеть** <br>


Для работы кода необходимо предоставить доступ к Google Disk, создать в корне папки Autocorrelation и Lables, загрузить в них набор данных и метки, а также создать в корне папку multilayer_perceptron для сохранения результатов обучения

Загрузка данных с Google Disk

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!ls "/content/drive/My Drive/"

 2_5305779684380783213.docx   IMG_20180624_012413.jpg
 baza.pdf		      IMG_20180624_012420.jpg
'Colab Notebooks'	      IMG_20180624_012444.jpg
 DSCF3993.jpg		      IMG_20180624_012451.jpg
 DSCF4073.jpg		      IMG_20180624_012500.jpg
 DSCF4121.jpg		      IMG_20180624_012501.jpg
 DSCF4264.jpg		      IMG_20180624_013405.jpg
 DSCF4275.jpg		      IMG_20191218_123350.jpg
 DSCF4971.jpg		      IMG_20191218_123418.jpg
 IMG_0689.jpg		      IMG_20191218_124720.jpg
 IMG_0823.jpg		      IMG_20191218_130138.jpg
 IMG_0878.jpg		      Lektsia_1_2022.ppt
 IMG_0954.jpg		      Lektsia_2_2022.ppt
 IMG_20180623_215438.jpg      Lektsia_3_2022.ppt
 IMG_20180623_215449.jpg      otvety_1.docx
 IMG_20180623_220917.jpg      otvety_2.docx
 IMG_20180623_224551.jpg      otvety_3.docx
 IMG_20180623_224604.jpg      Transmetropolitan
 IMG_20180623_224606.jpg      ULA_6552.gdoc
 IMG_20180623_224923.jpg      ULA_6675.gdoc
 IMG_20180623_225647.jpg      ULA_6679.gdoc
 IMG_20180623_225943.jpg      ULA_6681.gdoc
 IMG_2018062

Подключение библиотек

In [4]:
!pip install minisom

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for minisom: filename=MiniSom-2.3.1-py3-none-any.whl size=10589 sha256=9e52c11e939b4fd7d5fbf51c482805a3111278638fb642b0964330c4f9c2c127
  Stored in directory: /root/.cache/pip/wheels/c7/92/d2/33bbda5f86fd8830510b16aa98c8dd420129b5cb24248fd6db
Successfully built minisom


In [5]:
import matplotlib
matplotlib.use("Agg")
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.models import load_model
from keras.layers.core import Dense
from keras.optimizers import Adagrad
from keras.optimizers import Adadelta
from keras.optimizers import RMSprop
from keras.optimizers import Adam
from keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
from minisom import MiniSom
import pathlib
import pandas
import random
import pickle
import cv2
import os

from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

Преобразование набора данных в формат, удобный для обучения. Эта часть кода может быть выполнена один раз, поскольку в нем преобразованные данные сохраняются на Google Disk, после чего они могут быть загружены в уже готовом виде

In [11]:
au_p = pathlib.Path("/content/drive/My Drive/Colab Notebooks/Autocorrelation/").glob('**/*')
AutocorPaths = [x for x in au_p if x.is_file()]
random.seed(40)
la_p = pathlib.Path("/content/drive/My Drive/Colab Notebooks/Labels/").glob('**/*')
LabelsPaths = [x for x in la_p if x.is_file()]
print(len(AutocorPaths))
Paths = list(zip(AutocorPaths, LabelsPaths))
random.shuffle(Paths)
#AutocorPaths, LabelsPaths = zip(*temp)
#AutocorPaths, LabelsPaths = list(AutocorPaths), list(LabelsPaths)

data = []
labels = []


for autocorpath, labelspath in Paths:
  auto_file = open(autocorpath,'r')
  label_file = open(labelspath,'r') 

  for auto_line in auto_file:
    auto_temp = auto_line.split()
    autocor_sample = [float(item) for item in auto_temp]

    label_line = label_file.readline()
    label_temp = label_line.split()
    label_sample = [int(item) for item in label_temp]

    # Функция автокорреляции записывается столько раз, сколько меток ей сопоставлено
    for label in label_sample:
      one_hot_label = [0, 0, 0, 0, 0]
      one_hot_label[label-1] = 1
      labels.append(one_hot_label)
      data.append(autocor_sample)
  

  label_file.close()
  auto_file.close()
    

print(len(labels))

data = np.array(data)
labels = np.array(labels)

with open("/content/drive/My Drive/Colab Notebooks/hybrid_network/data.pickle", 'wb') as f:
  pickle.dump(data, f)
print("Data saved")

with open("/content/drive/My Drive/Colab Notebooks/hybrid_network/labels.pickle", 'wb') as f:
  pickle.dump(labels, f)
print("Labels saved")

1
745
Data saved
Labels saved


Преобразование набора данных в обучающую и тестовую выборки

In [12]:
with open("/content/drive/My Drive/Colab Notebooks/hybrid_network/data.pickle", 'rb') as f:
  data = pickle.load(f)
print("Data loaded")

with open("/content/drive/My Drive/Colab Notebooks/hybrid_network/labels.pickle", 'rb') as f:
  labels = pickle.load(f)
print("Labels loaded")

(trainX, testX, trainY, testY) = train_test_split(data, labels,
                                                  test_size=0.15,
                                                  random_state=40)

#(trainX, testX, train_labels_nums, test_labels_nums) = train_test_split(data, labels_nums,
#                                                  test_size=0.15,
#                                                  random_state=42)

print("Dataset prepared")

Data loaded
Labels loaded
Dataset prepared


Создание архитектуры многослойного перцептрона

In [13]:
model = Sequential() 
model.add(Dense(30,input_shape=(30,), activation='sigmoid'))
model.add(Dense(12, activation='sigmoid'))
model.add(Dense(5, activation='softmax'))

INIT_LR = 0.01
opt = Adam(learning_rate=INIT_LR)

model.compile(loss="categorical_crossentropy",optimizer=opt,
              metrics=["categorical_accuracy"])

print ("Model compiled")

Model compiled


Обучение многослойного перцептрона. Используется параметр checkpointer, позволяющий после обучения сохранить нейронную сеть, показавшую наилучший результат. Также может быть использован параметр early_stopping, останавливающий обучение, если в течение заданного количества эпох не наблюдалось в качестве сети

In [14]:
EPOCHS = 1000

# checkpointer = ModelCheckpoint(filepath='/content/drive/My Drive/Colab Notebooks/hybrid_network/EasyNet_{epoch:3d}_{categorical_accuracy:.3f}.model', verbose=1, save_freq=10)
checkpointer = ModelCheckpoint(filepath='/content/drive/My Drive/Colab Notebooks/hybrid_network/EasyNet_best.model', verbose=1, save_best_only=True)
# checkpointer = ModelCheckpoint(filepath='/content/drive/My Drive/Colab Notebooks/hybrid_network/point.model', verbose=1)
early_stopping = EarlyStopping(monitor='val_categorical_accuracy', patience=200, verbose=1)


# model.fit(x_train, y_train, batch_size=128, epochs=20, verbose=0, validation_data=(X_test, Y_test), callbacks=[checkpointer])


H = model.fit(trainX, trainY, validation_data=(testX,testY), epochs=EPOCHS, batch_size=32, callbacks = [checkpointer])
# H = model.fit(trainX, trainY, validation_data=(testX,testY), epochs=EPOCHS, batch_size=32, callbacks = [early_stopping])
#H = model.fit(trainX, trainY, validation_data=(testX,testY), epochs=EPOCHS, batch_size=128)

print("Model trained")

Epoch 1/1000
 1/20 [>.............................] - ETA: 7s - loss: 1.6877 - categorical_accuracy: 0.1875
Epoch 1: val_loss improved from inf to 1.45627, saving model to /content/drive/My Drive/Colab Notebooks/hybrid_network/EasyNet_best.model
20/20 [==============================] - 2s 65ms/step - loss: 1.5675 - categorical_accuracy: 0.3033 - val_loss: 1.4563 - val_categorical_accuracy: 0.5804
Epoch 2/1000
 1/20 [>.............................] - ETA: 0s - loss: 1.4289 - categorical_accuracy: 0.6562
Epoch 2: val_loss improved from 1.45627 to 1.23738, saving model to /content/drive/My Drive/Colab Notebooks/hybrid_network/EasyNet_best.model
20/20 [==============================] - 1s 67ms/step - loss: 1.3717 - categorical_accuracy: 0.5845 - val_loss: 1.2374 - val_categorical_accuracy: 0.5446
Epoch 3/1000
18/20 [==========================>...] - ETA: 0s - loss: 1.1633 - categorical_accuracy: 0.5781
Epoch 3: val_loss improved from 1.23738 to 1.04668, saving model to /content/drive/My Dr

Построение графика зависимости качества обучения многослойного перцептрона от количества прошедших эпох. График, как и модель, полученная на последней эпохе, сохраняются в Google Disk в папке hybrid_network. <br>
В случае использования при обучении параметра early_stopping для корректного построения графика необходимо раскомментировать первую строчку кода и закомментировать вторую

In [15]:
#N = np.arange(0, early_stopping.stopped_epoch+1)
N = np.arange(0, EPOCHS)
plt.clf()
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="vall_loss")
plt.plot(N, H.history["categorical_accuracy"], label="train_acc")
plt.plot(N, H.history["val_categorical_accuracy"], label="val_acc")
plt.title("Results")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy/")
plt.legend()
plt.savefig("/content/drive/My Drive/Colab Notebooks/hybrid_network/Loss.png")
model.save("/content/drive/My Drive/Colab Notebooks/hybrid_network/EasyNet.model")

print("End")

End


Результаты обучения нейронной сети с использованием лучшей полученной модели. Выводятся результаты применения сети к тестовой выборке и отчет о качестве работы сети. В отчете представлены значения метрик precision, recall и f1-score для каждого из пяти типовых аварийных состояний судна, а также их средние значения для всех типов вместе

In [16]:
model = load_model("/content/drive/My Drive/Colab Notebooks/hybrid_network/EasyNet_best.model")
predictions = model.predict (testX, batch_size = 32)
for i in predictions:
  print(i)
#print(predictions)

print(classification_report(testY.argmax(axis=1),
                            predictions.argmax(axis=1), target_names=("1 wreck type","2 wreck type","3 wreck type","4 wreck type","5 wreck type")))

4/4 [==============================] - 0s 3ms/step
[4.1314471e-03 4.6270012e-04 3.2111906e-02 4.3581736e-01 5.2747661e-01]
[0.10737009 0.8136464  0.03194696 0.03330392 0.01373262]
[0.01953409 0.09728681 0.10837161 0.5819702  0.1928373 ]
[0.11275471 0.78365827 0.03831865 0.04649741 0.0187709 ]
[2.4901896e-03 4.2464570e-04 2.4928243e-01 1.9316138e-01 5.5464137e-01]
[0.8125705  0.00092242 0.15338881 0.02385355 0.00926471]
[1.0688164e-02 9.2502014e-04 9.7569805e-01 8.3763888e-03 4.3123839e-03]
[3.6369227e-03 7.8330288e-04 9.8154318e-01 9.4341161e-03 4.6025808e-03]
[0.00412584 0.00128921 0.9860918  0.0057319  0.00276131]
[1.9416466e-03 2.5822734e-04 1.4411512e-01 2.9566455e-01 5.5802035e-01]
[0.27894002 0.37395397 0.24017447 0.07680593 0.03012555]
[1.4160697e-03 3.5439862e-04 2.1362051e-01 2.1639468e-01 5.6821442e-01]
[0.0008901  0.00074658 0.15893605 0.23238721 0.60704005]
[0.00381312 0.00119163 0.00332859 0.35604855 0.63561803]
[0.09899466 0.8263217  0.02723601 0.03382166 0.01362596]
[9.1

Преобразование one hot представления меток в целые числа от 1 до 5 и обучение сети Кохонена

In [17]:
train_labels_nums = [];
test_labels_nums = [];

for one_hot_label in trainY:
  res = [idx for idx, val in enumerate(one_hot_label) if val != 0]
  label = res[0]+1
  train_labels_nums.append(label)

for one_hot_label in testY:
  res = [idx for idx, val in enumerate(one_hot_label) if val != 0]
  label = res[0]+1
  test_labels_nums.append(label)

koh_train_x = model.predict (trainX, batch_size = 32)
koh_test_x = model.predict (testX, batch_size = 32)
som = MiniSom(x= 1, y = 5, input_len = 5, neighborhood_function = 'gaussian', sigma=0.1, learning_rate=0.001)
# som.random_weights_init([testX[0],testX[1],testX[14],testX[3],testX[5]])
# centers = [koh_train_x[25], koh_train_x[110], koh_train_x[14], koh_train_x[0], koh_train_x[114]]
som.random_weights_init(koh_train_x)
starting_weights = som.get_weights().copy()
som.train(koh_train_x, 1000000)

qnt = som.quantization(koh_test_x)
som.quantization_error(koh_test_x)


4/4 [==============================] - 0s 4ms/step


0.15509768955914427

Результаты обучения нейронной сети. Выводятся результаты применения сети к тестовой выборке и отчет о качестве работы сети. В отчете представлены значения метрик precision, recall и f1-score для каждого из пяти типовых аварийных состояний судна, а также их средние значения для всех типов вместе


In [18]:
winmap = som.labels_map(koh_train_x, train_labels_nums)
default_class = np.sum(list(winmap.values())).most_common()[0][0]
result = []
for d in koh_train_x:
    win_position = som.winner(d)
    if win_position in winmap:
        result.append(winmap[win_position].most_common()[0][0])
    else:
        result.append(default_class)
print(classification_report(train_labels_nums, result))

              precision    recall  f1-score   support

           1       0.66      0.82      0.73       128
           2       0.73      0.93      0.82       122
           3       0.94      0.39      0.55       122
           4       0.73      0.28      0.40       130
           5       0.55      0.92      0.69       131

    accuracy                           0.67       633
   macro avg       0.72      0.67      0.64       633
weighted avg       0.72      0.67      0.64       633

